LangChain错误容错机制
目前有两种容错机制
+ 错误重试
    RetryOutputParser
+ 错误修复
    OutputFixingParser

In [1]:
import os
from dotenv import load_dotenv

# 加载 .env 文件中的环境变量
load_dotenv(override=True)  # 使用 override=True 确保加载最新的 .env 数据

True

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model=os.environ.get("OPENAPI_MODEL"),
    base_url=os.environ.get("OPENAPI_API_BASE"),
    api_key=os.environ.get("OPENAPI_API_KEY"),
    temperature=0,
)

question = "LangChain是什么？"

## 错误重试

In [10]:
from langchain.output_parsers import RetryOutputParser
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.exceptions import OutputParserException
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field

template = """Based on the user question, provide an Action and Action Input for what step should be taken.
{format_instructions}
Question: {query}
Response:"""


class Action(BaseModel):
    action: str = Field(
        ..., description="The action to take, should be one of [Search, Lookup, None]"
    )
    action_input: str = Field(..., description="The input to the action")


parser = PydanticOutputParser(pydantic_object=Action)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# 假设用户的输入合成提示值
prompt_value = prompt.format_prompt(query="北京今天天气如何?")

# 假设得到一个错误回答，不符合pydantic字段要求，缺少action_input字段
bad_response = '{"action": "search"}'

# 运行抛出异常
try:
    output = parser.parse(bad_response)
except OutputParserException as e:
    print(f"Caught an OutputParserException: {e}")

    # 使用RetryOutputParser进行错误重试
    retry_parser = RetryOutputParser.from_llm(
        parser=parser,
        # 定义使用哪个模型进行重试，这里继续使用之前定义的llm
        llm=llm,
        max_retries=3,
    )
    output = retry_parser.parse_with_prompt(bad_response, prompt_value)
    print(f"Retry succeeded with output: {output}")

Caught an OutputParserException: Failed to parse Action from completion {"action": "search"}. Got: 1 validation error for Action
action_input
  Field required [type=missing, input_value={'action': 'search'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 
Retry succeeded with output: action='Search' action_input='北京今天天气'


## 错误修复

In [14]:
from langchain.output_parsers import OutputFixingParser
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.exceptions import OutputParserException
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field


class Actor(BaseModel):
    name: str = Field(..., description="The name of the actor")
    film_names: list[str] = Field(
        ..., description="The names of the films the actor has appeared in"
    )


parser = PydanticOutputParser(pydantic_object=Actor)

actor_query = "Generate the filmography for a random actor."

# 假设得到一个错误回答，这个是json格式，不符合pydantic格式
bad_response = '{"name": "Tom Hanks", "film_names": ["Forrest Gump"]}'

# 运行抛出异常
try:
    output = parser.parse(bad_response)
    print(output)
except OutputParserException as e:
    print(f"Caught an OutputParserException: {e}")

    # 使用OutputFixingParser进行错误修复
    new_parser = OutputFixingParser.from_llm(
        parser=parser,
        # 定义使用哪个模型进行重试，这里继续使用之前定义的llm
        llm=llm,
    )
    output = new_parser.parse(bad_response)
    print(f"Fixing succeeded with output: {output}")

name='Tom Hanks' film_names=['Forrest Gump']
